**``sentinel 8 imgs 500x500 m around poi B1 - B11 bands L-1C(TOA) imgs this time.``**

## Notes

- dataset_v0 

In [1]:
# ! pip install psycopg2-binary
# ! pip install postgres

# # # ! sudo apt install libpq-dev python3-dev 
# # ! sudo apt-get install libpq-dev python-dev -y --fix-missing


# ! sudo apt-get install postgresql-server-dev-all -y
# ! sudo apt-get install postgresql-common -y


# ! pip install datacube rioxarray odc-cloud
# ! pip install geopandas --upgrade

# this shit ain't working!

# Load metadata and dependencies

## Connect with earth engine

In [2]:
import ee
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
service_account = user_secrets.get_secret("google_service_account")
json_key_path = '/kaggle/input/gc-json-key/ticktickbloom-project-000e9745b830-json-key.json'
credentials = ee.ServiceAccountCredentials(service_account, json_key_path)

ee.Initialize(credentials)

# Import data laoding and manipulating libs

In [3]:
# ! pip install geemap -q

In [4]:
import datetime
import os
import sys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import joblib
from joblib import Parallel, delayed

import geopy.distance as distance

import warnings
warnings.filterwarnings('ignore')

In [5]:
metadata = pd.read_csv('/kaggle/input/ticktickbloomdataset/metadata.csv')
sub_format = pd.read_csv('/kaggle/input/ticktickbloomdataset/submission_format.csv')
train_labels = pd.read_csv('/kaggle/input/ticktickbloomdataset/train_labels.csv')

In [6]:
metadata.date = pd.to_datetime(metadata.date)
metadata.sort_values(by='date', ascending=False)

,uid,latitude,longitude,date,split
6540,hfvr,36.796200,-121.782000,2021-12-29,test
6864,howu,36.708500,-121.749000,2021-12-29,test
12443,nsoi,36.736800,-121.734000,2021-12-29,test
14254,prfi,36.751800,-121.742000,2021-12-29,test
17559,thki,36.725400,-121.730000,2021-12-29,test
...,...,...,...,...,...
5317,fwbt,44.850500,-93.515700,2013-01-04,train
5566,gdxr,44.877646,-93.557842,2013-01-04,train
6144,guny,44.878889,-93.490833,2013-01-04,train
13644,paev,44.822478,-93.367962,2013-01-04,train


# Define config 

In [7]:
SATELLITE_COLLECTION = 'COPERNICUS/S2_HARMONIZED'


img_folder = 'sentinel-l1c_500m_v0'

# img_metadata_cols = ['CLOUD_COVER', 'CLOUD_COVER_LAND', 'IMAGE_QUALITY_OLI', 'IMAGE_QUALITY_TIRS', 'DATE_ACQUIRED']
img_metadata_cols = ['CLOUDY_PIXEL_PERCENTAGE']

# sentinel l1c (toa) bands
BANDS = [
    'B1',
    'B2',
    'B3',
    'B4',
    'B5',
    'B6',
    'B7',
    'B8',
    'B8A',
    'B9',
    'B10',
    'B11',
    'B12'
        ]

# Collect data

In [8]:
def get_bufferbox(lat: float, long: float, buffer: float = 500, lat_first: bool = False):
    distance_search = distance.distance(meters=buffer)  # 0.5 km around our point of intrest
    min_lat = distance_search.destination((lat, long), bearing=180)[0]
    min_long = distance_search.destination((lat, long), bearing=270)[1]
    max_lat = distance_search.destination((lat, long), bearing=0)[0]
    max_long = distance_search.destination((lat, long), bearing=90)[1]
    if lat_first == True:
        # not the way for ee
        return (min_lat, min_long, max_lat, max_long)     # for ee.Geometry.Rectangle
    else:
        return (min_long, min_lat, max_long, max_lat)


def get_date_n_days_ago(cur_date: str, n_days: int =30) -> str:
    return str(cur_date - datetime.timedelta(days=n_days)).split(' ')[0]


def get_img_col_in_daterange(
    cur_date: str,
    wrs_path: int = None,
    wrs_row: int = None,
    lat_long: tuple=None, 
    get_neigbh_img: bool = False,   # retrys to get neighbouring imgs (frow wrs path and row) around the box to avoid "clip error."
    bands: list = BANDS
    )-> ee.ImageCollection:
    
    """returns img collection of 15 imgs from cur_date to cur_date -16 days contanning given (lat, long)"""
    
    if get_neigbh_img == False:
        lat, lng = lat_long
        img_col = ee.ImageCollection(SATELLITE_COLLECTION
          ).filterDate(
            start=get_date_n_days_ago(pd.to_datetime(cur_date)),
            opt_end=str(cur_date)
          ).filterBounds(
              ee.Geometry.Point(lng, lat)
          ).select(
              bands
          ).limit(15)

    if get_neigbh_img == True:
        img_col = ee.ImageCollection(SATELLITE_COLLECTION
          ).filterDate(
            start=get_date_n_days_ago(pd.to_datetime(cur_date)),
            opt_end=str(cur_date)
          ).filter(
            ee.Filter.eq('WRS_PATH', wrs_path)
          ).filter(
            ee.Filter.eq('WRS_ROW', wrs_row)
          ).select(
              bands
          ).limit(15)

    return img_col  


def get_latest_img(img_col: ee.ImageCollection, uid: str = None) -> ee.Image:
    """returns latest image and it's metadata from collection of images"""
    img_metadata = []
    # sort this image collection
    if int(img_col.size().getInfo()) >= 1:
        # if atleast 1 img in img_col sort by date and get first img
        my_img = img_col.limit(1, 'system:time_start', False).first()
        
        for col in img_metadata_cols:
            img_metadata.append(my_img.get(col).getInfo())
        return my_img, img_metadata
    else:
        #  no images in collection
        return None, [np.nan]*len(img_metadata_cols)

  
# convert latest image into array of array bands 
def get_arr_col_from_img(img: ee.Image, img_metadata: list, lat_long: tuple=None) -> np.array:
    if img is not None:
        bands = img.bandNames().getInfo()
        bbox = get_bufferbox(lat_long[0], lat_long[1], lat_first=False)
        bbox = ee.Geometry.Rectangle(bbox)
        band_arrs = img.sampleRectangle(bbox)
        img_band_col = []
        for band in bands:
            band_arr = np.array(band_arrs.get(band).getInfo())
            if band_arr is not None:
                img_band_col.append(band_arr)
            else:
                img_band_col.append(None)  
                return None
    else:
        return None
        pass

    return np.array(img_band_col + img_metadata)  # return both img_vals and img_metadata in single write to optimize.


# save the band_arr 
def save_bands(band_col:np.array, uid: str) -> None:
    
    if os.path.exists(f'{img_folder}') and os.path.isdir(f'{img_folder}'):
        joblib.dump(filename=f'{img_folder}/{uid}.npy', value=band_col)
    else:
        os.makedirs(f'{img_folder}')
        joblib.dump(filename=f'{img_folder}/{uid}.npy', value=band_col)


def collect_data(uid: str, lat_long: tuple, cur_date:str)-> None:
    """
    1. get past date to search from
    2. get the ee.ImageCollection in that daterange and in given lat_long
    3. 
      3.1 if there is atleast one get latest img from the collection .
        3.1.1 Try: converting latest img to np. arrays
          ---> SAVE
        3.1.2 if clipping error Try: neigbh imgs
          ---> SAVE
        3.1.3 error in neigbh imgs append uid to error_imgs return
      3.2 else append uid to img_not_found and return
    """
    img_col = get_img_col_in_daterange(cur_date = cur_date, lat_long=lat_long)
    try :
        my_img, img_metadata = get_latest_img(img_col, uid)
        if my_img is not None: 
            wrs_path, wrs_row = my_img.get('WRS_PATH').getInfo(), my_img.get('WRS_ROW').getInfo()
            band_col = get_arr_col_from_img(img=my_img,img_metadata=img_metadata, lat_long=lat_long)
            save_bands(band_col, uid)
            # lazy_save_dict['uid'] = band_col
            # lazy_save_band_cols.append(band_col)
            # lazy_save_band_uids.append(uid)
    
        else:
          # add uid to img_not_found
          img_not_found.append(uid)
          # print('img is None')
          return None

    except Exception as e:
        ## retries to get req bbox from neighbouring images
        # retry_collect_data(uid=uid, lat_long=lat_long, cur_date=cur_date, wrs_path=wrs_path, wrs_row=wrs_row)
        print(e)


def retry_collect_data(uid: str, lat_long: tuple, cur_date:str, wrs_path: int, wrs_row: int) -> None:
    # print('retrying with ngbh imgs ...')
    ngbh_imgs = [
        (wrs_path+1, wrs_row-1), 
        (wrs_path+1, wrs_row), 
        (wrs_path, wrs_row-1), 
        (wrs_path, wrs_row+1), 
        (wrs_path-1, wrs_row), 
        (wrs_path-1, wrs_row-1)
    ]
    # iterate over negibouring img patches and try to get the bbox (to avoid clip error)
    for ngbh_img in ngbh_imgs:
        wrs_path, wrs_row = ngbh_img
        try:
            img_col = get_img_col_in_daterange(cur_date=cur_date, wrs_path=wrs_path, wrs_row=wrs_row, get_neigbh_img=True)
            my_img, img_metadata = get_latest_img(img_col, uid)
            if my_img is not None:
                band_col = get_arr_col_from_img(img=my_img, img_metadata=img_metadata, lat_long=lat_long)
                save_bands(band_col, uid)
                ## lazy save
                # lazy_save_dict['uid'] = band_col
                # lazy_save_band_cols.append(band_col)
                # lazy_save_band_uids.append(uid)
                break
            if (my_img is None) and (ngbh_img == ngbh_imgs[-1]):
                # if no img in collection and this is the last img_patch add it to img_not_found
                img_not_found.append(uid)
                return None

        except Exception as f:
            if (ngbh_img == ngbh_imgs[-1]):
                err_imgs.append(uid)
                errors.append(f)
                print(f)
          

In [9]:
def collect_data_wrapper(data: pd.Series, n: int):
    cur_date = str(data.loc[n, 'date']).split(' ')[0]  # only date needed
    uid = data.loc[n, 'uid']
    lat_long = tuple(data.loc[n][['latitude', 'longitude']])
    
    try:
        collect_data(cur_date=cur_date, uid=uid, lat_long=lat_long)
    except Exception as e:
        err_imgs.append(uid)
        errors.append(e)

In [10]:
%%time

dataset_avail_date = '2015-06-23'
# data = metadata.sample(20).reset_index(drop=True)
sorted_data = metadata.sort_values(by='date')
data = sorted_data[sorted_data.date >= dataset_avail_date].reset_index(drop=True)
data = data[10_000:].reset_index(drop=True)

# img_metadata_df = pd.DataFrame(columns=img_metadata_cols) # df to hold gathered imgs' metadata
err_imgs = []         # img found but failed to extract bbox arrays!
errors = []           # corresponding errors of err_imgs
img_not_found = []    # No single img in img_col!!

lazy_save_dict = {}

lazy_save_band_cols = []
lazy_save_band_uids = []

print("total samples:", data.shape[0])
# allmissed imgs = err_imgs + img_notfound

total samples: 8367
CPU times: user 10.8 ms, sys: 976 µs, total: 11.8 ms
Wall time: 11.6 ms


In [11]:
# %%time

# #  loop through whole dataset and collect imgs : 
# for n in tqdm(range(data.shape[0]), desc='Collecting Imgs: ', colour='green'):
#     collect_data_wrapper(data, n)

In [12]:
%%time

print(data.shape[0])

# parrllelize shit
_ = Parallel(n_jobs=-1,backend='threading')(
    delayed(collect_data_wrapper)
    (data, n)
    for n in tqdm(range(data.shape[0]), desc='Collecting Imgs: ', colour='green')
)

8367


Image.sampleRectangle: Fully masked pixels / pixels outside of the image footprint when sampling band 'B2' with no default value set. Note that calling sampleRectangle() on an image after ee.Image.clip() may result in a sampling bounding box outside the geometry passed to clip().


Image.sampleRectangle: Fully masked pixels / pixels outside of the image footprint when sampling band 'B12' with no default value set. Note that calling sampleRectangle() on an image after ee.Image.clip() may result in a sampling bounding box outside the geometry passed to clip().


Image.sampleRectangle: Fully masked pixels / pixels outside of the image footprint when sampling band 'B5' with no default value set. Note that calling sampleRectangle() on an image after ee.Image.clip() may result in a sampling bounding box outside the geometry passed to clip().


Image.sampleRectangle: Fully masked pixels / pixels outside of the image footprint when sampling band 'B1' with no default value set. Note that calling sampleRectangle() on an image after ee.Image.clip() may result in a sampling bounding box outside the geometry passed to clip().


...

Image.sampleRectangle: Fully masked pixels / pixels outside of the image footprint when sampling band 'B2' with no default value set. Note that calling sampleRectangle() on an image after ee.Image.clip() may result in a sampling bounding box outside the geometry passed to clip().


Image.sampleRectangle: Fully masked pixels / pixels outside of the image footprint when sampling band 'B1' with no default value set. Note that calling sampleRectangle() on an image after ee.Image.clip() may result in a sampling bounding box outside the geometry passed to clip().


Image.sampleRectangle: Fully masked pixels / pixels outside of the image footprint when sampling band 'B12' with no default value set. Note that calling sampleRectangle() on an image after ee.Image.clip() may result in a sampling bounding box outside the geometry passed to clip().


Image.sampleRectangle: Fully masked pixels / pixels outside of the image footprint when sampling band 'B11' with no default value set. Note that calling sampleRectangle() on an image after ee.Image.clip() may result in a sampling bounding box outside the geometry passed to clip().


CPU times: user 15min 1s, sys: 50.5 s, total: 15min 52s
Wall time: 2h 51min 35s


In [13]:
err_imgs

[]

In [14]:
errors

[]

In [15]:
img_not_found

[]

In [16]:
print("misses:", len(img_not_found) + len(err_imgs))
print('Total files pulled:', len(os.listdir(f'/kaggle/working/{img_folder}')))

misses: 0
Total files pulled: 8197


In [17]:
# save files
joblib.dump(err_imgs, 'missed_imgs-p2.joblib')
joblib.dump(img_not_found, 'imgs_not_found_in_col-p2.joblib')
joblib.dump(errors, 'errors-p2.joblib')

# img_metadata_df.to_csv(f'{img_folder}/{img_folder}_metadata.csv', index=False)

['errors-p2.joblib']

In [18]:
os.cpu_count()

4